In [8]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_excel('WaitData.Published.xlsx')
df

,Wait,SumHowEarlyWaiting,AvgHowEarlyWaiting,LineCount0Strict,SumWaits,LineCount0,LineCount1,LineCount2,LineCount3,LineCount4,...,NumAddOnsToday,NumAddOnsLastW2,NumScheduledNextSlot,NumScheduledNextW2,SumTimeToCompleteNextSlot,SumTimeToCompleteNextW2,WithContrastCountWaiting,WithAndWithoutContrastCountWaiting,WithContrastCountInProgress,WithAndWithoutContrastCountInProgress
0,0,372,62,6,249,6,5,6,6,4,...,4,2,0,0,0,0,3,1,1,0
1,-3,335,67,3,195,5,4,6,6,5,...,4,1,2,0,32,0,2,1,2,0
2,-25,274,68,4,150,4,5,6,4,6,...,4,1,1,1,13,16,1,1,3,0
3,36,369,73,2,179,5,6,4,5,5,...,3,1,2,1,45,16,3,1,1,0
4,-21,438,73,5,192,6,5,5,6,5,...,3,0,2,1,32,15,4,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23578,38,239,59,0,115,4,4,1,1,0,...,0,0,1,1,14,17,3,0,0,0
23579,8,174,58,3,39,3,1,0,0,0,...,0,0,2,1,33,13,2,0,0,0
23580,12,122,61,2,33,2,1,0,0,0,...,0,0,2,1,34,13,1,0,0,0
23581,2,78,78,0,19,1,1,0,0,0,...,0,0,2,2,40,33,1,0,0,0


In [9]:
# task 5

X = df.drop('Wait', axis=1)
Y = df['Wait'].tolist()

from sklearn import linear_model
model = linear_model.LinearRegression()
model.fit(X, Y)
Ypred = model.predict(X) # use trained regression model to predict
r = Y-Ypred # compute prediction error (residual)
e = abs(r).mean()

e


23.16628893634705

In [11]:
# task 6

if True: # just in case I want to disable this part
    print('\n>Python feature selection:')
    from sklearn.feature_selection import RFE
    from itertools import compress
    for nFeatures in range(1,4):
        rfe = RFE(model, n_features_to_select=nFeatures)
        X_rfe = rfe.fit_transform(X,Y) #transforming data using RFE   
        #Fitting the data to model
        model.fit(X_rfe,Y)
        #print(rfe.support_)
        #print(rfe.ranking_)
        cols = list(compress( X.columns, rfe.support_))
        model.fit(X[cols],Y)
        e = abs(Y-model.predict(X[cols])).mean()
        print(e, cols)



>Python feature selection:
32.50541172120655 ['CardiacCount']
31.224937706982736 ['CardiacCount', 'DelayedInLine']
31.25752894868932 ['CardiacCount', 'StartTime', 'DelayedInLine']


In [29]:
#task 7

def get_best_n_cols(n):
    best_cols = []

    for i in range(n):
        best_col = None
        best_error = float('inf')
        for col in X.drop(best_cols, axis=1).columns:
            
            X_tmp = X[best_cols + [col]]
            model = linear_model.LinearRegression()
            model.fit(X_tmp, Y)
            Ypred = model.predict(X_tmp) # use trained regression model to predict
            r = Y-Ypred # compute prediction error (residual)
            e = abs(r).mean()

            if e < best_error:
                best_col = col
                best_error = e
        best_cols.append(best_col)
        
    return best_cols, best_error

get_best_n_cols(3)

(['DelayedInLine', 'LineCount0Strict', 'LineCount0'], 28.69649063225338)

In [31]:
#task 8

for n in range(1, 14):
    if get_best_n_cols(n)[1] < 24:
        print(n)
        break

10


In [38]:
#task 9

from tqdm import tqdm

results = []

for idx1, col1 in tqdm(enumerate(X.columns), total=len(X.columns)):
    for idx2, col2 in enumerate(X.drop(col1, axis=1).columns[idx1 + 1:]):
        for col3 in X.drop([col1, col2], axis=1).columns[idx2 + 1:]:
            columns = [col1, col2, col3]
            X_tmp = X[columns]
            model = linear_model.LinearRegression()
            model.fit(X_tmp, Y)
            Ypred = model.predict(X_tmp) # use trained regression model to predict
            r = Y-Ypred # compute prediction error (residual)
            e = abs(r).mean()

            results.append([columns, e])

sorted(results, key=lambda x: x[1])[0]

100%|██████████| 85/85 [14:02<00:00,  9.92s/it]


[['SumDelayWaitingByExamCode', 'NumCompletedToday', 'AheadCount'],
 26.072745167091224]